# Install

In [1]:
!pip install -q tensorflow

import tensorflow as tf
from tensorflow.keras import layers
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print("TF version:", tf.__version__)

TF version: 2.19.0


# Load Dataset

In [2]:
data = fetch_california_housing()
X = StandardScaler().fit_transform(data.data)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = tf.cast(X_train, tf.float32)
y_train = tf.cast(y_train, tf.float32)

# Custom Activation, Initializer, Regularizer, Constraint

In [3]:
class ClipConstraint(tf.keras.constraints.Constraint):
    def __call__(self, w): return tf.clip_by_value(w, -1.0, 1.0)

class L1Penalty(tf.keras.regularizers.Regularizer):
    def __call__(self, x): return 0.001 * tf.reduce_sum(tf.abs(x))

def scaled_init(shape, dtype=None): return tf.random.normal(shape, stddev=0.05, dtype=dtype)
def param_relu(x): return tf.where(x > 0, x, 0.1 * x)

# Custom Loss

In [4]:
class SmoothL1(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        diff = tf.abs(y_true - y_pred)
        return tf.where(diff < 1.0, 0.5 * diff**2, diff - 0.5)

# Custom Metric

In [5]:
class MAEMetric(tf.keras.metrics.Metric):
    def __init__(self, name="mean_absolute_error", **kwargs):
        super().__init__(name=name, **kwargs)
        self.total = self.add_weight(name="total", initializer="zeros")
        self.count = self.add_weight(name="count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        err = tf.abs(y_true - y_pred)
        self.total.assign_add(tf.reduce_sum(err))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self): return self.total / self.count

# Custom Model (ResidualBlock style)


In [6]:
class ResidualRegressor(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = layers.Dense(64, activation=param_relu,
                               kernel_initializer=scaled_init,
                               kernel_regularizer=L1Penalty(),
                               kernel_constraint=ClipConstraint())
        self.d2 = layers.Dense(64)
        self.out = layers.Dense(1)
    def call(self, x):
        x1 = self.d1(x)
        x2 = self.d2(x1)
        return self.out(x1 + x2)

# Custom Optimizer

In [7]:
class SimpleSGD(tf.Module):
    def __init__(self, learning_rate=0.01):
        self.lr = learning_rate
    def apply_gradients(self, grads, vars):
        for g, v in zip(grads, vars):
            v.assign_sub(self.lr * g)

# Custom Training Loop

In [8]:
model = ResidualRegressor()
loss_fn = SmoothL1()
optimizer = SimpleSGD(learning_rate=0.01)
metric = MAEMetric()

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(32)

# train loop
for epoch in range(3):
    metric.reset_state()
    for xb, yb in train_ds:
        with tf.GradientTape() as tape:
            preds = model(xb, training=True)
            loss = loss_fn(yb, tf.squeeze(preds))
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(grads, model.trainable_variables)
        metric.update_state(yb, tf.squeeze(preds))
    print(f"Epoch {epoch+1} - MAE: {metric.result().numpy():.4f}")

Epoch 1 - MAE: 0.6233
Epoch 2 - MAE: 0.4973
Epoch 3 - MAE: 0.4810
